# Inference from Torch of angiograms   



In [ ]:
1+1

In [4]:
import torch
import torch.nn as nn
import onnx
import dill as pickle
import os
import matplotlib.pyplot as plt  # Add this line

In [ ]:
torch.__version__

In [ ]:
# Check if running on Mac OS
is_mac = os.name == 'posix' and os.uname().sysname == 'Darwin'
print('posix' if os.name == 'posix' else 'not posix')
print('mac' if is_mac else 'not mac')


In [7]:
# Set up paths
is_mac = os.name == 'posix' and os.uname().sysname == 'Darwin'
rootPath = "~/Projects/AWI/NetExploration/" if is_mac else '/mnt/SliskiDrive/AWI/AWIBuffer/' # '/Volumes/Crucial X8/AWIBuffer/'


In [ ]:
rootPath

In [9]:
# model_path = "/home/ubuntu/U-Mamba-Adjustment/data/nets/UXlstmBot-nnUNetPlans_2d-DC_and_CE_loss-w-1-20-20-dill.pth"
model_path = rootPath + "UXlstmBot-nnUNetPlans_2d-DC_and_CE_loss-w-1-20-20-dill.pth"
model_path = "/Users/billb/Projects/AWI/NetExploration/UXlstmBot-nnUNetPlans_2d-DC_and_CE_loss-w-1-20-20-dill.pth"


In [ ]:
model_path

In [ ]:
# Set up device
gpuDevice = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {gpuDevice}") 

In [12]:
import sys
sys.path.append("/Users/billb/github/nnUNet-Adjustment")

In [13]:
import nnunetv2

In [14]:
# !pip install blosc2

In [ ]:
model = torch.load(model_path, pickle_module=pickle, map_location=gpuDevice)
# model.to(gpuDevice)
model.eval()

## Test Model with HDF5 Input

In [ ]:
import platform

system = platform.system()
if "Darwin" in system:
    if os.path.isdir("/Volumes/Crucial X8"):
        dataDir = "/Volumes/Crucial X8/AWIBuffer"
    else:
        dataDir = "/Users/billb/Projects/AWI/NetExploration"
elif "Linux" in system:
    dataDir = "/mnt/SliskiDrive/AWI/AWIBuffer"
else:
    dataDir = None  # or some default path

# dataDir = "/home/ubuntu/data"
angiogramH5Path = dataDir + "/AngiogramsDistilledUInt8List.h5"
angiogramH5Path

In [ ]:
import h5py

# Open the HDF5 file and print all dataset keys
with h5py.File(angiogramH5Path, 'r') as f:
    # Get all keys at root level
    keys = list(f.keys())
    print("Dataset keys in HDF5 file:")
    for key in keys:
        print(f"- {key}")


In [ ]:
# Load first angiogram from HDF5 file
import random
with h5py.File(angiogramH5Path, 'r') as f:
    # Get first key
    hdfKey = random.choice(keys)
    print(f"Loading dataset: {hdfKey}")
    # Load data into tensor
    agram = torch.from_numpy(f[hdfKey][:]).float()
    print(f"Loaded tensor shape: {agram.shape}")


In [ ]:
#Display the 30th frame of the angiogram
plt.imshow(agram[30], cmap='gray')
plt.colorbar()
plt.show()


In [ ]:
# Normalize angiogram by subtracting mean and dividing by standard deviation
xagram = (agram - agram.mean()) / agram.std()
print(f"Normalized tensor shape: {xagram.shape}")


In [ ]:
# Create input tensor with 5 consecutive frames centered around frame 30
start_idx = 28  # 30-2 to get 2 frames before
end_idx = 33    # 30+3 to get 2 frames after (exclusive)
z = xagram[start_idx:end_idx].unsqueeze(0)  # Add batch dimension
print(f"Input tensor shape: {z.shape}")


In [25]:
# Move model and input tensor to GPU device
# gpuDevice = 'mps'
model = model.to(gpuDevice)
z = z.to(gpuDevice)


In [26]:
with torch.no_grad():
    model.eval()
    y = model(z)


In [ ]:
# Apply softmax along dimension 1 (second dimension) which has size 3
y = torch.nn.functional.softmax(y, dim=1)
print(f"Output tensor shape after softmax: {y.shape}")


In [ ]:
# Display the 3rd channel (index 2) of the output
plt.imshow(y[0, 2].cpu().detach().numpy(), cmap='gray')
plt.colorbar()
plt.title('Output Channel 3')
plt.show()


In [ ]:
# Calculate number of valid frame groups (each group has 5 consecutive frames)
num_frames = xagram.shape[0]
num_groups = num_frames - 4  # Each group needs 5 frames

# Create tensor to hold all valid frame groups
z5 = torch.zeros((num_groups, 5, 512, 512))

# Fill z5 with overlapping groups of 5 consecutive frames
for i in range(num_groups):
    z5[i] = xagram[i:i+5]

print(f"Shape of tensor containing all valid 5-frame groups: {z5.shape}")


In [ ]:
# Get the middle 10 frames from z5
middle_idx = z5.shape[0] // 2  # Find middle index
start_idx = middle_idx - 5    # 10 frames before middle
end_idx = middle_idx + 5      # 10 frames after middle
z5 = z5[start_idx:end_idx]     # Keep only middle 20 frames

print(f"Shape of tensor after selecting middle 20 frames: {z5.shape}")


In [31]:
# Feed z5 into the model and get the output
model = model.to(gpuDevice)
z5 = z5.to(gpuDevice)

with torch.no_grad():
    model.eval()
    y5 = model(z5)


In [ ]:
y5.shape

In [ ]:
# Apply softmax along dimension 1 (second dimension) which has size 3
ys5 = torch.nn.functional.softmax(y5, dim=1)
print(f"Output tensor shape after softmax: {ys5.shape}")


In [ ]:
# Display the 3rd channel (index 2) of batch member 35
plt.imshow(ys5[5, 2].cpu().detach().numpy(), cmap='gray')
plt.colorbar()
plt.title('Output Channel 3 - Batch 35')
plt.show()


## Export Back to ONNX

# Export model back to ONNX
onnxOutputPath = model_path.replace(".pth", ".onnx")


In [35]:
# Export model back to ONNX
onnxOutputPath = model_path.replace(".pth", ".onnx")

In [ ]:

# Move both model and input tensor to CPU for export
# model_for_export = modelPerOnnx.to(gpuDevice)
# input_for_export = z5.to(gpuDevice)

# with torch.inference_mode():
#     torch.onnx.export(model,
#                      z,
#                      onnxOutputPath, 
#                      export_params=True,
#                      #opset_version=18, 
#                      do_constant_folding=True,
#                      verbose=True,
#                      input_names=['input'],
#                      output_names=['output'], 
#                      dynamic_axes={'input': {0: 'batch_size'}, 
#                                  'output': {0: 'batch_size'}}, 
#                      training=torch.onnx.TrainingMode.EVAL)
torch.onnx.export(model,
                     z,
                     onnxOutputPath, 
                     export_params=True,
                     #opset_version=18, 
                     do_constant_folding=True,
                     verbose=True)



In [ ]:
# onnx_program = torch.onnx.export(model, z, dynamo=True)
onnx_program = torch.onnx.dynamo_export(model, z)

In [ ]:
# with torch.inference_mode():
#     onnx_program = torch.onnx.dynamo_export(model,
#                      z, 
#                      export_params=True,
#                      #opset_version=18, 
#                      do_constant_folding=True,
#                      verbose=True,
#                      input_names=['input'],
#                      output_names=['output'], 
#                      dynamic_axes={'input': {0: 'batch_size'}, 
#                                  'output': {0: 'batch_size'}}, 
#                      training=torch.onnx.TrainingMode.EVAL)

In [29]:
onnx_program.save(onnxOutputPath)

## Save and Load PyTorch Model

In [ ]:
# Save PyTorch model
torchModelPath = onnxPath.replace(".onnx", "-torch-onnx.pt")
torch.save(modelPerOnnx, torchModelPath)

# Load and verify
checkModel = torch.load(torchModelPath, weights_only=False)
checkModel.eval()
with torch.inference_mode():
    result = checkModel(random_tensor)

print("Verification output shape:", result.shape)

## TorchScript Conversion

In [ ]:
# Create example inputs with different batch sizes
example_input_1 = torch.randn(2, 5, 512, 512).to(gpuDevice)
example_input_2 = torch.randn(4, 5, 512, 512).to(gpuDevice)

# Create traced model
tracedModel = torch.jit.trace(model, example_input_1, check_trace=False)

# Save traced model
# tracedModelPath = model_path.replace(".pth", "-torchscript-traced-onnx.pt")
# tracedModelperOnnx.save(tracedModelPath)

## Verify Traced Model

In [ ]:
# Load and test traced model
# tracedModel = torch.jit.load(tracedModelPath)
tracedModel = tracedModel.to(gpuDevice)

tracedModel.eval()
with torch.inference_mode():
    result = tracedModel(z5)

print("Traced model output shape:", result.shape)

# Test with different batch size
# result = tracedModelperOnnx(example_input_2)
# print("Different batch size output shape:", result.shape)

In [ ]:
# Export model back to ONNX
onnxOutputPath = model_path.replace(".pth", ".onnx")
onnxOutputPath

In [ ]:
torch.onnx.export(tracedModel,
                     z,
                     onnxOutputPath, 
                     export_params=True,
                     #opset_version=18, 
                     do_constant_folding=True,
                     verbose=True)

In [ ]:
from typing import Tuple
import torch

def get_torch_cuda_versions() -> Tuple[str, str, bool]:
    """
    Get PyTorch and CUDA versions along with CUDA availability status.

    Returns
    -------
    Tuple[str, str, bool]
        A tuple containing:
        - PyTorch version (str)
        - CUDA version (str) if available, 'N/A' if not
        - CUDA availability status (bool)
    """
    torch_version = torch.__version__
    cuda_available = torch.cuda.is_available()
    cuda_version = torch.version.cuda if cuda_available else "N/A"
    
    return torch_version, cuda_version, cuda_available

def print_versions() -> None:
    """
    Print PyTorch and CUDA version information to console.
    """
    torch_version, cuda_version, cuda_available = get_torch_cuda_versions()
    
    print(f"PyTorch Version: {torch_version}")
    print(f"CUDA Available: {cuda_available}")
    print(f"CUDA Version: {cuda_version}")

if __name__ == "__main__":
    print_versions()